In [2]:
pip install joblib


Note: you may need to restart the kernel to use updated packages.


In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Title
st.title("Athlete Medal Prediction App")

# File Upload
uploaded_file = st.file_uploader("Upload CSV file", type=["csv"])

def preprocess_data(df):
    # Drop irrelevant columns
    df = df.drop(columns=["ID", "Name"], errors='ignore')
    
    # Handle missing values
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    numerical_imputer = SimpleImputer(strategy='median')
    
    categorical_columns = df.select_dtypes(include=['object']).columns
    numerical_columns = df.select_dtypes(exclude=['object']).columns
    
    df[categorical_columns] = categorical_imputer.fit_transform(df[categorical_columns])
    df[numerical_columns] = numerical_imputer.fit_transform(df[numerical_columns])
    
    # Encode categorical variables
    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
    
    # Scale numerical features
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    
    return df, label_encoders, scaler

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.write("Dataset Preview:", df.head())
    
    if "Medal" in df.columns:
        df["Medal"] = df["Medal"].fillna("None")
        df["Medal"] = df["Medal"].apply(lambda x: 1 if x != "None" else 0)
        
        X = df.drop(columns=["Medal"], errors='ignore')
        y = df["Medal"]
        
        # Preprocess Data
        X, encoders, scaler = preprocess_data(X)
        
        # Split Data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Train Model
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        
        # Evaluate Model
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        st.write(f"Model Accuracy: {accuracy:.2f}")
        
        # Save model
        joblib.dump(model, "athlete_medal_model.pkl")
        joblib.dump(encoders, "label_encoders.pkl")
        joblib.dump(scaler, "scaler.pkl")
        
        st.success("Model Trained and Saved Successfully!")
        
        # Prediction Section
        st.subheader("Predict Medal for New Athletes")
        input_data = {}
        
        for col in X.columns:
            input_data[col] = st.number_input(f"{col}", value=float(X[col].mean()))
        
        if st.button("Predict"):
            input_df = pd.DataFrame([input_data])
            input_df[X.columns] = scaler.transform(input_df[X.columns])
            prediction = model.predict(input_df)[0]
            result = "Won a Medal" if prediction == 1 else "No Medal"
            st.write(f"Prediction: {result}")
            


2025-03-19 12:47:47.738 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


2025-02-11 21:29:12.599 
  [33m[1mWarning:[0m to view this Streamlit app on a browser, run it with the following
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
    